<a href="https://colab.research.google.com/github/maddarauci/speech_recognition/blob/main/Deep_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk 
nltk.download('punkt')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

import json 
import pickle 

import numpy as np

from keras.models import Sequential 
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD 
import random

words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
  for pattern in intent['patterns']:
    # word tokenization
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    # adding documents
    documents.append((w, intent['tag']))

    # adding classes to the classes list 
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), 'Documents')
print(len(classes), 'Classes')
print(len(words), 'unique lemmatized words', words)

pickle.dump(words,open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# initializing the training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
  #initializing the the bag of words
  bag = []
  # list of tokenized words for the pattern
  pattern_words = doc[0]
  # lemmatize each word - create base word, in attempt to represent related words 
  pattern_words =  [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
  # create our bag of words array with 1, if word match found in current pattern 
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

  # output is a '0' for each tag anf '1' for current tag (for each pattern)
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1

  training.append([bag, output_row]) 

# shuffle out features and turn into np.array 
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("training data created")

# create model - 3 layers. first layer 128 neurons, second 64 and 3rd output layer contains number of number 
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(.05))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# compile the model. stochastic gradient descent with Nesterov accelerated gradient gives good results for this model 
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist) # h5 is a data saving file format 


print('\nmodel created')





In [ ]:
pip install tkinter

ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
  return sentence_words 

def bow(sentence, words, show_details=True):
  #tokenize the pattern
  sentence_words = clean_up_sentence(sentence)
  # baf of words - matrix of N words, vocab matrix
  bag = [0]*len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w == s:
        # assign 1 if current word is in the vocab position
        bag[i] = 1
        if show_details:
          print('found in bag: %s' % w)
    return(np.array(bag))

def predict_class(sentence, model):
  # filter out prediction below a threshold 
  p = bow(sentence, words, show_details=False)
  res = model.predict(np.array([p]))[0]
  ERROR_THRESHOLD = 0.25
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append({'intent': classes[r[0]], 'probability': str([1])})
  return return_list

def getResponse(ints, intents_json):
  tag = ints[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    if(i['tag']== tag):
      result = random.choice(i['response'])
      break
  return result

def chatbot_response(msg):
  ints = predict_class(msg, model)
  res = getResponse(ints, intents)
  return res 

# creating the gui 
import tkinter as tk
from tkinter import * 
import os, sys 

if os.environ.get('DISPLAY','') == '':
    print('no display found. Using :0.0')
    os.environ.__setitem__('DISPLAY', ':0.0')

def send():
  msg = EntryBox.get('1.0', 'end-lc').strip()
  EntryBox.delete('0.0', END)

  if msg != '':
    ChatLog.config(state=NORMAL)
    ChatLog.insert(END, "U: " + msg + '\n\n')
    ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

    res = chatbot_response(msg)
    ChatLog.insert(END, "Bot: " + res + '\n\n')

    ChatLog.config(state=DISABLED)
    ChatLog.yview(END)

#root = Tkinter.Tk()
root = tk.Tk()
root.title("Feet to Meters")

root.geometry('400,500')
root.resizeable(width=False, height=False)

# creating window
ChatLog = Text(root, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

# bind scrollbar to chat window 
scrollbar = Scrollbar(root, command=ChatLog.yview, cursor='heart')
ChatLog['yscrollcommand'] = scrollbar.set

# create btn to send msg 
SendButton = Button(root, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send)

# create the box to enter msg 
EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)

#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

root.mainloop()

print('\ndone')

no display found. Using :0.0


TclError: ignored